In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import lasso_path, enet_path
import matplotlib.pyplot as plt
from itertools import cycle
from urllib.parse import urlparse

import mlflow
import mlflow.sklearn

In [2]:
def plot_enet_descent_path(X, y, l1_ratio):
    # Compute paths
    eps = 5e-3  # the smaller it is the longer is the path

    # Reference the global image variable
    global image
    
    print("Computing regularization path using ElasticNet.")
    alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio, fit_intercept=False)

    # Display results
    fig = plt.figure(1)
    ax = plt.gca()

    colors = cycle(['b', 'r', 'g', 'c', 'k'])
    neg_log_alphas_enet = -np.log10(alphas_enet)
    for coef_e, c in zip(coefs_enet, colors):
        l1 = plt.plot(neg_log_alphas_enet, coef_e, linestyle='--', c=c)

    plt.xlabel('-Log(alpha)')
    plt.ylabel('coefficients')
    title = 'ElasticNet Path by alpha for l1_ratio = ' + str(l1_ratio)
    plt.title(title)
    plt.axis('tight')

    # Display images
    image = fig
    
    # Save figure
    fig.savefig("ElasticNet-paths.png")

    # Close plot
    plt.close(fig)

    # Return images
    return image

In [51]:
csv_url = '/data/winequality-red.csv'
data = pd.read_csv(csv_url)

In [55]:
def train_wine_quality(data, in_alpha, in_l1_ratio):
    # Evaluate metrics
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    warnings.filterwarnings("ignore")
    np.random.seed(40)
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)
    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop('quality',axis=1)
    test_x = test.drop('quality', axis=1)
    train_y = train["quality"]
    test_y = test["quality"]

    if float(in_alpha) is None:
        alpha = 0.05
    else:
        alpha = float(in_alpha)

    if float(in_l1_ratio) is None:
        l1_ratio = 0.05
    else:
        l1_ratio = float(in_l1_ratio)
    
    mlflow.set_experiment("MLflow Customer Demo")

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(lr, "model",registered_model_name="demo-wine-pyspark-model")
        
#         mlflow.sklearn.save_model(lr, "/data/lr_wine_quality")

        # Call plot_enet_descent_path
#         plot_enet_descent_path(data.values, np.squeeze(np.asarray(data[["quality"]].values)), l1_ratio)

        # Log artifacts (output files)
#         mlflow.log_artifact("ElasticNet-paths.png")
#         mlflow.log_artifact("/data/winequality-red.csv")

In [56]:
train_wine_quality(data, 0.06, 0.045)

Elasticnet model (alpha=0.060000, l1_ratio=0.045000):
  RMSE: 0.6995652065911567
  MAE: 0.53553674162548
  R2: 0.30658994966819797


Successfully registered model 'demo-wine-pyspark-model'.
2022/04/28 13:06:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: demo-wine-pyspark-model, version 1
Created version '1' of model 'demo-wine-pyspark-model'.
